In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score

from google.colab import files
uploaded = files.upload()

# Функция вычисления accuracy
def compute_accuracy(preds, labels):
    preds_labels = torch.argmax(preds, dim=1)
    return accuracy_score(labels, preds_labels)

# Функция оценки модели
def evaluate_model(trainer, test_dataset):
    predictions, labels, _ = trainer.predict(test_dataset)
    predictions = torch.from_numpy(predictions)
    acc = compute_accuracy(predictions, labels)
    print(f'Accuracy on test dataset: {acc:.4f}')

# Предобработка данных
data = pd.read_csv(r"organization_reviews.csv", index_col=0)
data['rating'] = data['rating'] - 1
data['rating'] = data['rating'].astype('int8')

# Разделение датасэта на тренировочный и тестовый наборы
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['text'], data['rating'], test_size=0.2, random_state=42
)

class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/sbert_large_mt_nlu_ru", num_labels=5)

train_dataset = ReviewDataset(train_texts, train_labels, tokenizer, max_length=128)
val_dataset = ReviewDataset(val_texts, val_labels, tokenizer, max_length=128)

# Параметры обучения
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Обучение модели с помощью Trainer
trainer.train()

# Оценка модели на тестовом наборе
evaluate_model(trainer, val_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_mt_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,No log,1.649340


TrainOutput(global_step=81, training_loss=1.6931493311752508, metrics={'train_runtime': 2260.1294, 'train_samples_per_second': 0.287, 'train_steps_per_second': 0.036, 'total_flos': 75487205587968.0, 'train_loss': 1.6931493311752508, 'epoch': 1.0})